In [1]:
import requests
import pandas as pd
import scipy.stats as stat
from datetime import date
from config import gkey
from citipy import citipy

import json
import gmaps

In [2]:
dfCities = pd.read_csv('Cities.csv')

In [3]:
gmaps.configure(api_key = gkey)

location = dfCities[['Latitude', 'Longitude']]
humidity = dfCities['Humidity %']

fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(location, weights=humidity, max_intensity = dfCities['Humidity %'].max())

fig.add_layer(heatmap)

In [4]:
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
dfGoodCities = dfCities.loc[(dfCities['Temperature (F)'] >=70) & (dfCities['Temperature (F)'] < 80) & 
                            (dfCities['Wind Speed (mph)'] < 6) &
                            (dfCities['Cloudiness %'] < 5)]
dfGoodCities

,City,Temperature (F),Humidity %,Cloudiness %,Wind Speed (mph),Latitude,Longitude,Country,Date
9,kapaa,70.77,83,1,4.70,22.08,-159.32,US,04/10/20
99,tura,73.81,35,0,4.54,25.52,90.22,IN,04/10/20
202,narasannapeta,79.23,84,0,5.59,18.42,84.05,IN,04/10/20
258,kahului,73.78,73,1,4.25,20.89,-156.47,US,04/10/20
291,baracoa,78.91,75,0,4.14,20.35,-74.50,CU,04/10/20
476,crestview,71.08,35,1,5.82,30.76,-86.57,US,04/10/20


In [10]:
params = {'type': 'lodging',
          'keyword': 'hotel',
          'radius': 5000,
          'key': gkey,
          'language': 'en'}

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

pins = []
for index, row in dfGoodCities.iterrows():
    params['location'] = f'{row["Latitude"]},{row["Longitude"]}'
    
    hotelJson = requests.get(url, params = params).json()
    try:
        latlon = (hotelJson['results'][0]['geometry']['location']['lat'], hotelJson['results'][0]['geometry']['location']['lng'] )

        city = row['City']
        name = hotelJson['results'][0]['name']
        country = row['Country']

        pins.append({'latlon': latlon,
                 'city': city,
                 'name': name,
                 'country': country})
    except:
        print(f'No Hotel found for {row["City"]}')
    

In [11]:
locations = [pin['latlon'] for pin in pins]

#stole this bit from https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#drawing-markers-lines-and-polygons
infoTemplate = """
<dl>
<dt>Hotel Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
#------------

markerInfo = [infoTemplate.format(**pin) for pin in pins]

markerLayer = gmaps.marker_layer(locations, info_box_content=markerInfo)
fig.add_layer(markerLayer)
fig

Figure(layout=FigureLayout(height='420px'))